In [1]:
from matplotlib import pyplot as plt
import math
import numpy as np
import pandas as pd
import xarray as xr
import glob
from cmip6_preprocessing.preprocessing import combined_preprocessing
import cftime
from datetime import datetime
xr.set_options(display_style='html')
plt.rcParams['figure.figsize'] = 8,3

import gsw as gs
from IPython.display import clear_output

In [2]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

We calculate density, buoyancy frequency, and isopycnal slope from T and S.
Below uses the paths where my data are and checks to make sure we have a list of models for which I have T and S data.

In [5]:
data_dir = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip6/CONCATTED/'
data_dir_so = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_so/cmip6/'


files_c6 =  glob.glob(data_dir+ 'thetao_Oyr_*_piControl*.nc')
files_so_c6 =  glob.glob(data_dir_so+ 'so_Oyr_*_piControl*.nc')


models=[]
for file in files_c6:
    model = file.split("_")[5]
    models.append(model)
MODELS=np.unique(models)


models=[]
for file in files_so_c6:
    model = file.split("_")[5]
    models.append(model)
MODELS2=np.unique(models)
 
MODELS_do_c6 = intersection(MODELS, MODELS2)


In [6]:
data_dir = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip5/CONCATTED/'
data_dir_so = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_so/cmip5/'

files_c5 =  glob.glob(data_dir+ 'thetao_*yr_*_piControl*.nc')
files_so_c5 =  glob.glob(data_dir_so+ 'so_*yr_*_piControl*.nc')

models=[]
for file in files_c5:
    model = file.split("_")[5]
    models.append(model)
MODELS=np.unique(models)


models=[]
for file in files_so_c5:
    model = file.split("_")[5]
    models.append(model)
MODELS2=np.unique(models)

MODELS_do_c5 = intersection(MODELS, MODELS2)


Below chooses a random model (CESM2) to grab a z-grid to interpolate to and 

In [9]:
file = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip6/CONCATTED/thetao_Oyr_CESM2_piControl_r1i1p1f1_gn_regridded_050001-059912.nc'

dc  = xr.open_dataset(file).assign_attrs(source_id='CESM2')
dc=combined_preprocessing(dc)
Tc= dc['thetao']
Tc = Tc.isel(time = slice(40,80)).mean('time')
if Tc.dims[0]!='lev':
    Tc = Tc.rename({Tc.dims[0]:'lev'})   
TcX=Tc.assign_coords(model=model)

Next calculate metrics salinity, density, N$^2$ (called "stratification"), or $\frac{\partial\sigma_2}{\partial y}$ and $\frac{\partial\sigma_2}{\partial z}$ for isopycal slope <br>
Below could be written as a loop over each metric, but it was long for a notebook and I liked using it interactively

In [ ]:
p=2000
averaging_period_start=0
averaging_period_end=80
g=9.8
rho_0 = 1028
lat_to_meters = 1/(111*1000)


density = 0
stratification = 1
slopes = 0

cmip6=0
cmip5=1

if cmip6:
    MODELS_do = MODELS_do_c6
    files = files_c6
    files_so = files_so_c6
    ens = 'cmip6'
else:
    MODELS_do = MODELS_do_c5
    files = files_c5
    files_so = files_so_c5
    ens='cmip5'


S=[]
ST=[]
SZ=[]
SY=[]
PX=[]

for count,model in enumerate(MODELS_do):
    print(model)
    
    files_c = [match for match in files if model in match]
    files_s = [match for match in files_so if model in match]
           
    file = files_c[0]
    file_s = files_s[0]
    
    dc  = xr.open_dataset(file).assign_attrs(source_id=model)
    dc=combined_preprocessing(dc)
    variable = list(dc.keys())[-1] #grab variable name for temp (not all the same)
    Tc= dc[variable].rename('thetao')#rename to the same variable name in data array
    Tc = Tc.isel(time = slice(averaging_period_start,averaging_period_end)).mean('time')
    Tc=Tc.where(np.abs(Tc)<1e3) #here I mask where some models have weird fill values
    Tc = Tc.rename({Tc.dims[0]:'lev'}) #here is because not all models have the same vertical coord name
    Tcx=Tc.assign_coords(model=model)
        
    ds  = xr.open_dataset(file_s).assign_attrs(source_id=model)
    ds=combined_preprocessing(ds)
    Sc= ds['so']
    Sc = Sc.isel(time = slice(averaging_period_start,averaging_period_end)).mean('time')
    Sc=Sc.where(np.abs(Tc)<1e3)
    Sc = Sc.rename({Sc.dims[0]:'lev'}) 
    Scx=Sc.assign_coords(model=model)
        
 
    SA = gs.SA_from_SP(Scx,p,Scx.x,Scx.y)
    CT = gs.CT_from_t(SA,Tcx,p)
    Pc = gs.density.sigma2(SA,CT)

    if stratification: 
        strat = (g/rho_0)*Pc.differentiate("lev")
        strat = strat.assign_coords(model=model)
        strat = strat.interp_like(TcX)*TcX/TcX
        ST.append(strat)

    if slopes:
        #slope = drhody/drhodz, but it's smoother to take zonal mean of each first so we save seperately
        drhodz = Pc.differentiate("lev")
        drhody= Pc.differentiate("y")*lat_to_meters
        drhodz = drhodz.assign_coords(model=model)
        drhodz = drhodz.interp_like(TcX)*TcX/TcX
        drhody = drhody.assign_coords(model=model)
        drhody = drhody.interp_like(TcX)*TcX/TcX
        SY.append(drhodz)
        SZ.append(drhody)

    if density:
        Pcx = Pc.interp_like(TcX)*TcX/TcX
        Pcx = Pcx.assign_coords(model=model)
        PX.append(Pcx)
  
clear_output(wait=True)
    
    
#below,save drho/dx, drho/dz seperately so we can look at the slope in different basins (not ultimately used in paper)
if slopes: 
    drhody_control_model=xr.concat(SY,dim='model',coords='minimal',compat='override')
    drhodz_control_model=xr.concat(SZ,dim='model',coords='minimal',compat='override')
    drhodz_control_model_sig2.to_netcdf('drhodz_control_model_'+ens+'.nc')
    drhody_control_model_sig2.to_netcdf('drhody_control_model_'+ens+'.nc')
    drhodz_control_model_sig2.close()
    drhody_control_model_sig2.close()
    
if stratification:
    strat_control_model_sig2=xr.concat(ST,dim='model',coords='minimal',compat='override')
    strat_control_model_sig2.to_netcdf('strat_control_model_sig2_'+ens+'.nc')
    strat_control_model_sig2.close()

if density:
    Dense_control_model_sig2=xr.concat(PX,dim='model',coords='minimal',compat='override')
    Dense_control_model_sig2.to_netcdf('Dense_control_model_sig2_'+ens+'.nc')
    Dense_control_model_sig2.close()